In [101]:

using Markdown
using InteractiveUtils

using PlutoUI# Allows to create table of contents and sliders

using Plots, LaTeXStrings

using DifferentialEquations

using ForwardDiff, LinearAlgebra

using Roots



In [102]:
macro bind(def, element)
    #! format: off
    quote
        local iv = try Base.loaded_modules[Base.PkgId(Base.UUID("6e696c72-6542-2067-7265-42206c756150"), "AbstractPlutoDingetjes")].Bonds.initial_value catch; b -> missing; end
        local el = $(esc(element))
        global $(esc(def)) = Core.applicable(Base.get, el) ? Base.get(el) : iv(el)
        el
    end
    #! format: on
end

@bind (macro with 1 method)

In [103]:
TableOfContents()

Plots.default(
	size = 600 .* (√2, 1), # A4 size
	dpi = 180, # density of pixels per inch
	linewidth = 3, margins = 5Plots.mm,
	label = false
); Plots.scalefontsizes(1.2)

In [104]:
begin
	function plotvectorfield(xs, ys, g; plotkwargs...)
	    fig = plot()
	    plotvectorfield!(fig, xs, ys, g; plotkwargs...)
	    return fig
	end
	
	function plotvectorfield!(figure, xs, ys, g; rescale = 1, plotkwargs...)
	
		xlims = extrema(xs)
		ylims = extrema(ys)
		
		N, M = length(xs), length(ys)
		xm = repeat(xs, outer=M)
		ym = repeat(ys, inner=N)
		
		field = g.(xm, ym)
		
		scale = rescale * (xlims[2] - xlims[1]) / min(N, M)
		u = @. scale * first(field)
		v = @. scale * last(field)
		
		steadystates = @. (u ≈ 0) * (v ≈ 0)
		
		u[steadystates] .= NaN
		v[steadystates] .= NaN
		
		z = (x -> √(x'x)).(field)
		
	    quiver!(
	        figure, xm, ym;
	        quiver = (u, v), line_z=repeat(z, inner=4),
	        xlims = xlims, ylims = ylims,
	        c = :batlow, colorbar = false,
	        plotkwargs...
	    )
	
	end
end


plotvectorfield! (generic function with 1 method)

In [105]:
Base.@kwdef struct Model
	b::Float64 = 0.65
	c::Float64 = 0.5
	ρ::Float64 = 0.03
end;

In [106]:


function f(x, l, model::Model)
	- 1\l - model.b * x + (x^2)/(x^2 + 1)
end;

function g(x, l, model::Model)
	model.ρ * l + 2*c*x + model.b * l - (2 * l * x)/(x^2 + 1)^2 
end;

function F!(du, u, model, t)
	du[1] = f(u[1], max(u[2], 0.), model)
	du[2] = g(u[1], max(u[2], 0.), model)

	return du
end;

field(x, a, model::Model) = F!(Vector{Float64}(undef, 2), [x, a], model, 0.);


function J(x, a, model::Model)
	l = -1/a
	u = [x, l]
	ForwardDiff.jacobian((du, u) -> F!(du, u, model, 0.), similar(u), u)
end;


In [107]:

xspan = (0., 4.); aspan = (0., 0.6);



### Dynamics of the system...

With a constant fertiliser use $a =$ $(@bind afig Slider(range(aspan...; step = 0.01), show_value = true))
"




### Nullclines

The nullclines are given by 



$\begin{align}
N_x &= \Big\{a = m(x)\Big\} \\
N_a &= \left\{(x, a): \sqrt{a} = \frac{\rho + m'(x)}{2cx}\right\}
\end{align}$





In [108]:

# ╔═╡ d917eda6-4265-408b-ad14-b421f2fb9b6f


function Nₓ(x, model:: Model) 
	model.b * x - (x^2)/(x^2 +1)
	
end; 

# ╔═╡ c2903662-fd60-48a6-850e-07f668b78f2a
function Nₐ(x, model::Model)
	num = (model.ρ + model.b - (2*x)/((x^2 + 1)^2))
	den = 2* model.c * x

	(ifelse(num > 0., num, NaN) / den)^2
end;


In [ ]:
# ╔═╡ d917eda6-4265-408b-ad14-b421f2fb9b6f


function Nₓ(x, model:: Model) 
	model.b * x - (x^2)/(x^2 +1)
	
end; 

# ╔═╡ c2903662-fd60-48a6-850e-07f668b78f2a
function Nₐ(x, model::Model)
	num = model.ρ + model.b - (2*x)/((x^2 + 1)^2)
	den = 2 * model.c * x

	(ifelse(num > 0., num, NaN) / den)^2
end;


In [109]:

# ╔═╡ 5e8ceff6-7388-429e-b8c6-a3a0ae4cef99
begin
	h = 1e-4
	
	function outofbounds(u, t, integrator)
		!all(0. .< u .< 5.)
	end
	
	callback = DiscreteCallback(outofbounds, terminate!)
	manifoldhorizon = 1000.
end;


In [110]:

"
- ``c =`` $(@bind c Slider(0.2:0.01:0.5, show_value = true, default = 0.5))
- ``\rho =`` $(@bind ρ Slider(0.01:0.01:0.5, show_value = true, default = 0.03))
- ``x_0 =`` $(@bind x₀ Slider(range(xspan...; step = 0.1), show_value = true, default = 2.))
"


"\n- ``c =`` Slider{Float64}(0.2:0.01:0.5, 0.5, true)\n- ``\rho =`` Slider{Float64}(0.01:0.01:0.5, 0.03, true)\n- ``x_0 =`` Slider{Float64}(0.0:0.1:4.0, 2.0, true)\n"

In [111]:

model = Model(c = c, ρ = ρ);



In [112]:

# ╔═╡ a43e8d20-cf23-4c52-86ff-5e5984067418
begin
	xspace = range(xspan...; length = 5001)

	intervalroots = find_zeros(x -> Nₓ(x, model) - Nₐ(x, model), xspan);

	nullclinefig = plot(xlims = xspan, ylims = aspan, xlabel = L"Phosphorus concentration $x$", ylabel = L"Fertiliser use $a$", title = L"Cost $c = %$c$")

	vline!(nullclinefig, [x₀]; c = :darkorange, linestyle = :dash)

	plot!(nullclinefig, xspace, x -> Nₓ(x, model); label = L"N_x", linestyle = :solid, c = :black)
	plot!(nullclinefig, xspace, x -> Nₐ(x, model); label = L"N_a", linestyle = :dot, c = :black)


	for x̄ in intervalroots
		ā = Nₓ(x̄, model)
		jac = J(x̄, ā, model)

		λ, v = eigen(jac)

		issaddle = all.(isreal(λ)) && prod(λ) < 0.

		c = issaddle ? :darkgreen : :darkred

		
		if issaddle
			ū = [x̄, ā]
			vₛ = v[:, λ .< 0] |> vec
			for u₀ in (ū - h * vₛ, ū + h * vₛ)
				ts = range(0, -manifoldhorizon; step = -0.1)
				prob = ODEProblem(F!, u₀, (0, -manifoldhorizon), model)
				mₛ = solve(prob, callback = callback)
	
				traj = mₛ(ts).u	
				xₜ = getindex.(traj, 1)
				λₜ = getindex.(traj, 2)
					
				plot!(nullclinefig, xₜ, λₜ; c = :darkgreen)
			end
		end

		scatter!(nullclinefig, [x̄], [ā]; c = c, markersize = 5, markerstrokewidth = 0)
	end
	
	nullclinefig
end



LoadError: MethodError: [0mCannot `convert` an object of type [92mSymbol[39m[0m to an object of type [91mFloat64[39m

[0mClosest candidates are:
[0m  convert(::Type{Float64}, [91m::Measures.AbsoluteLength[39m)
[0m[90m   @[39m [35mMeasures[39m [90mC:\Users\imaan\.julia\packages\Measures\PKOxJ\src\[39m[90m[4mlength.jl:12[24m[39m
[0m  convert(::Type{<:Real}, [91m::T[39m) where T<:SparseConnectivityTracer.AbstractTracer
[0m[90m   @[39m [36mSparseConnectivityTracer[39m [90mC:\Users\imaan\.julia\packages\SparseConnectivityTracer\aHOLr\src\overloads\[39m[90m[4mconversion.jl:10[24m[39m
[0m  convert(::Type{N}, [91m::D[39m) where {N<:Real, P, T, D<:SparseConnectivityTracer.Dual{P, T}}
[0m[90m   @[39m [36mSparseConnectivityTracer[39m [90mC:\Users\imaan\.julia\packages\SparseConnectivityTracer\aHOLr\src\overloads\[39m[90m[4mconversion.jl:31[24m[39m
[0m  ...


In [ ]:
let
	exfig = hline([0.]; c = :black, linestyle = :dash, xlabel = L"Phosphorus in the lake $x$", ylabel = L"\dot{x} = %$afig - m(x)")

	fex = x -> afig - m(x, model)
	plot!(exfig, xspace, fex; ylims = (-0.6, 0.6), c = :black, xlims = xspan)

	roots = find_zeros(fex, xspan)
	for x̄ in roots
		isstable = m′(x̄, model) > 0.
		scatter!(exfig, [x̄], [0.]; c = isstable ? :black : :white, markersize = 6) 
	end

	exfig
end

LoadError: UndefVarError: `afig` not defined

In [113]:
using Plots
using LinearAlgebra
using DifferentialEquations

# Define the nullcline functions
function nullcline_a(x, p, b, c)
    return (p + b - (2 * x) / ((x^2 + 1)^2)) / (2 * c * x)
end

function nullcline_x(b, x)
    return b * x - (x^2) / (x^2 + 1)
end

# Define the Jacobian matrix
function jacobian(x, l, b, c, p)
    df_dx = -b + (2 * x) / ((x^2 + 1)^2)
    df_dl = 0
    dg_dx = 2 * c - ((2 * l) * (1 - 3 * x^2)) / ((x^2 + 1)^3)
    dg_dl = p + b - (2 * x) / ((x^2 + 1)^2)
    return [df_dx df_dl; dg_dx dg_dl]
end

# Define the state and costate equations
function state_eq(l, b, x)
    return -1 / l - b * x + (x^2) / (x^2 + 1)
end

function costate_eq(l, b, c, p, x)
    return p * l + 2 * c * x + l * b - (2 * x * l) / (x^2 + 1)^2
end

# Define the system of ODEs
function system!(du, u, p, t)
    l, x = u
    b, c, p_val = p
    du[1] = state_eq(l, b, x)
    du[2] = costate_eq(l, b, c, p_val, x)
end

# Parameters
b = 0.65
p = 0.03
c = 0.5
x = range(0.01, 2, length=300)

# Compute nullclines
f2 = nullcline_a.(x, p, b, c)
f1 = nullcline_x.(b, x)

# Saddle points
ss1_x = 0.45301029168136897
ss1_a = 0.1241818769554022
ss2_x = 1.436961466047747
ss2_a = 0.2603043162259756
saddles = [(ss1_x, ss1_a), (ss2_x, ss2_a)]

# Plot the stable manifold
plot(x, f2, label="Costate nullcline, f₂(x)", lw=2)
plot!(x, f1, label="State nullcline, f₁(x)", lw=2)

for saddle in saddles
    J = jacobian(saddle[1], -1 / saddle[2], b, c, p)

    # Calculate eigenvalues and eigenvectors
    values, vectors = eigen(J)
    println("Eigenvalues: ", values)
    println("Eigenvectors: ", vectors)

    # Stable eigenvector (corresponding to the negative eigenvalue)
    stable_idx = argmin(abs.(values))
    println("Stable eigenvalue = ", values[stable_idx])
    stable_eigenvector = vectors[:, stable_idx]
    stable_eigenvector /= norm(stable_eigenvector)  # Normalize to unit length

    # Perturb the saddle point along the stable eigenvector
    h = 0.01
    z0_plus = [saddle[2], saddle[1]] .+ h * stable_eigenvector
    z0_minus = [saddle[2], saddle[1]] .- h * stable_eigenvector

    # Integrate backwards in time
    t_span = (-10.0, 0.0)
    t_eval = range(-10.0, 0.0, length=1000)
    prob_plus = ODEProblem(system!, z0_plus, t_span, (b, c, p))
    prob_minus = ODEProblem(system!, z0_minus, t_span, (b, c, p))
    sol_plus = solve(prob_plus, Tsit5(), saveat=t_eval)
    sol_minus = solve(prob_minus, Tsit5(), saveat=t_eval)

    # Plot the stable manifolds
    plot!(sol_plus[2, :], sol_plus[1, :], label="Stable Manifold (z₀ + h*vₛ)", color=:blue)
    plot!(sol_minus[2, :], sol_minus[1, :], label="Stable Manifold (z₀ - h*vₛ)", color=:red)
    scatter!([saddle[1]], [saddle[2]], label="Saddle Point", color=:black, markersize=4)
end

xlabel!("x")
ylabel!("a")
title!("Stable Manifold of the Saddle Point")
xlims!(0, 2)
ylims!(0, 2)
grid!(true)
legend!(fontsize=10)

LoadError: invalid redefinition of constant Main.J